In [ ]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch
import numpy as np

img = Image.open("csiro-biomass/test/ID1001187975.jpg")

model = timm.create_model(
    'vit_7b_patch16_dinov3.lvd1689m',
    pretrained=False,
    num_classes=0,  # remove classifier nn.Linear
    checkpoint_path="timm/vit_7b_patch16_dinov3.lvd1689m/model.safetensors"
)
model = model.eval().to(torch.bfloat16).to("cuda")

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)
transforms_train = timm.data.create_transform(**data_config, is_training=True)

In [3]:
transforms

Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(256, 256))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)

In [4]:
print(img.size)

#convert img to tensor
img_array = np.array(img)

print(img_array.shape)


(2000, 1000)
(1000, 2000, 3)


In [5]:
output = model(transforms(img_array).unsqueeze(0).to(torch.bfloat16).to("cuda"))  # output is (batch_size, num_features) shaped tensor

print(output.shape)

TypeError: Unexpected type <class 'numpy.ndarray'>